# Sandbox to use Python tools to check JSONs/YAMLs

In [1]:
## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import json
import jsonschema
import pathlib
import jsonref
import yaml

In [2]:
pathlib.Path.cwd()

PosixPath('/Users/jay/Desktop/biothings_explorer/jupyter notebooks/CX_WIPs/Draft_Metadata')

## checking the schemas for x-bte-associations

In [3]:
yaml_schema = pathlib.Path.cwd().joinpath("draft_x-bte-kgs-associations-schema.yaml")
with open(yaml_schema) as file:
    schema_from_yaml = yaml.load(file, Loader=yaml.SafeLoader)
    schema_from_yaml = json.dumps(schema_from_yaml, indent=2)
    schema_from_yaml = jsonref.loads(schema_from_yaml)

In [4]:
schema_from_yaml.keys()

dict_keys(['title', '$schema', 'type', 'description', 'properties', 'definitions'])

In [5]:
schema_from_yaml['properties'].keys()

dict_keys(['components'])

In [6]:
schema_from_yaml['properties']['components']

{'type': 'object',
 'properties': {'x-bte-kgs-associations': {'type': 'object',
   'patternProperties': {'.': {'type': 'object',
     'required': ['supportBatch',
      'inputs',
      'outputs',
      'predicate',
      'provenance',
      'numeric_measures_present',
      'category_measures_present'],
     'properties': {'supportBatch': {'type': 'boolean'},
      'inputSeparator': {'description': 'If this API supports multiple input IDs in one query, this is the symbol that needs to be between IDs',
       'type': 'string'},
      'inputs': {'description': 'Python-list-of-dictionaries-like, describes the identifier namespace of the inputs for the query.',
       'type': 'array',
       'items': {'type': 'object',
        'required': ['id', 'semantic'],
        'properties': {'id': {'description': 'ID namespace prefix for the query input',
          'type': 'string'},
         'semantic': {'description': 'Biolink entity class for the query input',
          'type': 'string'}}}},
     

In [7]:
json_schema = pathlib.Path.cwd().joinpath("draft_x-bte-kgs-associations-schema.json")
with open(json_schema) as file:
    schema_from_json = jsonref.load(file)

In [8]:
schema_from_json.keys()

dict_keys(['title', '$schema', 'type', 'description', 'properties', 'definitions'])

In [9]:
schema_from_json['properties'].keys()

dict_keys(['components'])

In [10]:
schema_from_json['properties']['components']

{'type': 'object',
 'properties': {'x-bte-kgs-associations': {'type': 'object',
   'patternProperties': {'.': {'type': 'object',
     'required': ['supportBatch',
      'inputs',
      'outputs',
      'predicate',
      'provenance',
      'numeric_measures_present',
      'category_measures_present'],
     'properties': {'supportBatch': {'type': 'boolean'},
      'inputSeparator': {'description': 'If this API supports multiple input IDs in one query, this is the symbol that needs to be between IDs',
       'type': 'string'},
      'inputs': {'description': 'Python-list-of-dictionaries-like, describes the identifier namespace of the inputs for the query.',
       'type': 'array',
       'items': {'type': 'object',
        'required': ['id', 'semantic'],
        'properties': {'id': {'description': 'ID namespace prefix for the query input',
          'type': 'string'},
         'semantic': {'description': 'Biolink entity class for the query input',
          'type': 'string'}}}},
     

In [11]:
schema_from_yaml == schema_from_json
## YAY that's exactly what should happen

True

## checking the non-ref single examples for x-bte-associations

In [12]:
yaml_nonrefexample = pathlib.Path.cwd().joinpath("draft_example-association.yaml")
with open(yaml_nonrefexample) as file:
    nonrefexample_from_yaml = yaml.load(file, Loader=yaml.SafeLoader)
#     nonrefexample_from_yaml = json.dumps(nonrefexample_from_yaml, indent=2)
#     nonrefexample_from_yaml = jsonref.loads(nonrefexample_from_yaml)

In [13]:
nonrefexample_from_yaml.keys()

dict_keys(['components'])

In [14]:
nonrefexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn'].keys()

dict_keys(['supportBatch', 'inputSeparator', 'parameters', 'requestBody', 'inputs', 'outputs', 'predicate', 'nodes_conflated', 'provenance', 'publications', 'numeric_measures_present', 'numeric_measures', 'category_measures_present'])

In [15]:
nonrefexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn']['nodes_conflated']

[{'original_type': 'Disease',
  'conflated_type': 'PhenotypicFeature',
  'where': 'DisGeNET'},
 {'original_type': 'Gene',
  'conflated_type': 'GeneProduct',
  'where': 'DisGeNET'}]

In [18]:
json_nonrefexample = pathlib.Path.cwd().joinpath("draft_example-association.json")
with open(json_nonrefexample) as file:
    nonrefexample_from_json = json.load(file)

In [19]:
nonrefexample_from_json['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn'].keys()

dict_keys(['supportBatch', 'inputSeparator', 'parameters', 'requestBody', 'inputs', 'outputs', 'predicate', 'nodes_conflated', 'provenance', 'publications', 'numeric_measures_present', 'numeric_measures', 'category_measures_present'])

In [20]:
nonrefexample_from_yaml == nonrefexample_from_json
## YAY that's exactly what should happen

True

## checking the ref single examples for x-bte-associations

In [21]:
yaml_refexample = pathlib.Path.cwd().joinpath("draft_example-association-w-refs.yaml")
with open(yaml_refexample) as file:
    refexample_from_yaml = yaml.load(file, Loader=yaml.SafeLoader)
    refexample_from_yaml = json.dumps(refexample_from_yaml, indent=2)
    refexample_from_yaml = jsonref.loads(refexample_from_yaml)

In [22]:
refexample_from_yaml['components'].keys()

dict_keys(['x-bte-kgs-associations', 'x-bte-reusable-info'])

In [23]:
refexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn'].keys()

dict_keys(['supportBatch', 'inputSeparator', 'parameters', 'requestBody', 'inputs', 'outputs', 'predicate', 'nodes_conflated', 'provenance', 'publications', 'numeric_measures_present', 'numeric_measures', 'category_measures_present'])

In [24]:
refexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn'] \
== nonrefexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn']
## YAYYAYAYAYAYAYAY

True

In [25]:
refexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn']['inputs']
refexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn']['outputs']
refexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn']['predicate']

[{'id': 'UMLS', 'semantic': 'Disease'}]

[{'id': 'NCBIGene',
  'semantic': 'Gene',
  'mapped_response_field': 'disgenet.gene_related_to_disease.gene_id'}]

{'biolink': 'related_to', 'id': 'SIO:001403', 'label': 'is_associated_with'}

In [26]:
refexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn']['nodes_conflated']

[{'original_type': 'Disease',
  'conflated_type': 'PhenotypicFeature',
  'where': 'DisGeNET'},
 {'original_type': 'Gene',
  'conflated_type': 'GeneProduct',
  'where': 'DisGeNET'}]

In [27]:
refexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn']['provenance']

[{'name': 'MyDisease.info API',
  'knowledge_source_type': 'service',
  'version': '2020-10-26',
  'method': 'ingest_consolidate'},
 {'name': 'DisGeNET',
  'knowledge_source_type': 'knowledgebase',
  'version': 'v7.0',
  'method': 'NLP_LHGDN',
  'method_info': {'url': ['https://www.disgenet.org/dbinfo#section11']}},
 {'name': 'GeneRIF',
  'knowledge_source_type': 'text',
  'version': '2009-03-31',
  'knowledge_source_context': {'taxon_specific': ['NCBITaxon:9606']}}]

In [28]:
refexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn']['numeric_measures']

[{'name': 'GDAscore',
  'standard_label': 'association_score',
  'value': 'disgenet.genes_related_to_disease.score',
  'range': {'minExclusive': 0, 'maxInclusive': 1},
  'numeric_direction_info': {'larger': 'more_evidence'},
  'measure_info': {'url': ['https://www.disgenet.org/dbinfo#section31']}},
 {'name': 'EI',
  'standard_label': 'evidence_index',
  'value': 'disgenet.genes_related_to_disease.EI',
  'range': {'minExclusive': 0, 'maxInclusive': 1},
  'numeric_direction_info': {'larger': 'more_consistent'},
  'measure_info': {'url': ['https://www.disgenet.org/dbinfo#section36']}},
 {'name': 'DSI',
  'standard_label': 'gene_specific_to_disease',
  'ontology_term': 'SIO:001351',
  'value': 'disgenet.genes_related_to_disease.DSI',
  'range': {'minExclusive': 0, 'maxInclusive': 1},
  'numeric_direction_info': {'larger': 'more_specific'},
  'measure_info': {'url': ['https://www.disgenet.org/dbinfo#section33']}},
 {'name': 'DPI',
  'standard_label': 'gene_specific_to_disease_class',
  'ont

## validate single examples against schema

In [29]:
jsonschema.validate(instance=refexample_from_yaml, schema=schema_from_yaml)
## YAYAYAYAYAY this means it validated!!!!!!!

In [30]:
jsonschema.validate(instance=nonrefexample_from_yaml, schema=schema_from_yaml)
## YAYAYAYAYAY this means it validated!!!!!!!

## try looking at a ton of examples

In [31]:
tons_of_associations = pathlib.Path.cwd().joinpath("draft_example-association-w-refs.yaml")
with open(tons_of_associations) as file:
    tonsoref = yaml.load(file, Loader=yaml.SafeLoader)
    tonsoref = json.dumps(tonsoref, indent=2)
    tonsoref = jsonref.loads(tonsoref)

In [32]:
len(tonsoref['components']['x-bte-kgs-associations'].keys())
tonsoref['components']['x-bte-kgs-associations'].keys()

21

dict_keys(['disease-gene1_disgenet-lhgdn', 'disease-gene2_disgenet-befree', 'disease-gene3_disgenet-hpo', 'disease-gene4_disgenet-uniprot', 'disease-gene5_disgenet-cgi', 'disease-gene6_disgenet-clinvar', 'disease-gene7_disgenet-gwascat', 'disease-gene8_disgenet-gwasdb', 'disease-gene9_disgenet-mgd', 'disease-gene10_disgenet-rgd', 'disease-gene11_disgenet-ctdmouse', 'disease-gene12_disgenet-ctdrat', 'disease-gene13_disgenet-clingen', 'disease-gene14_disgenet-genomics-eng', 'disease-gene15_disgenet-ctdhuman', 'disease-gene16_disgenet-psygenet', 'disease-gene17_disgenet-orphanet', 'gene-disease1_disgenet-lhgdn', 'gene-disease2_disgenet-befree', 'gene-disease3_disgenet-hpo', 'gene-disease4_disgenet-uniprot'])

In [33]:
tonsoref['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn'] == \
nonrefexample_from_yaml['components']['x-bte-kgs-associations']['disease-gene1_disgenet-lhgdn']
## YESSSSSSSSS

True

In [35]:
tonsoref['components']['x-bte-kgs-associations']['gene-disease2_disgenet-befree']

{'supportBatch': True,
 'inputSeparator': ',',
 'parameters': {'fields': 'disgenet.xrefs.umls, disgenet.genes_related_to_disease',
  'size': '250'},
 'requestBody': {'body': {'q': '{inputs[0]&source=BEFREE}',
   'scopes': 'disgenet.genes_related_to_disease.gene_id'},
  'header': 'application/x-www-form-urlencoded'},
 'inputs': [{'id': 'NCBIGene', 'semantic': 'Gene'}],
 'outputs': [{'id': 'UMLS',
   'semantic': 'Disease',
   'mapped_response_field': 'disgenet.xrefs.umls'}],
 'predicate': {'biolink': 'related_to',
  'id': 'SIO:001403',
  'label': 'is_associated_with'},
 'nodes_conflated': [{'original_type': 'Disease',
   'conflated_type': 'PhenotypicFeature',
   'where': 'DisGeNET'},
  {'original_type': 'Gene',
   'conflated_type': 'GeneProduct',
   'where': 'DisGeNET'}],
 'provenance': [{'name': 'MyDisease.info API',
   'knowledge_source_type': 'service',
   'version': '2020-10-26',
   'method': 'ingest_consolidate'},
  {'name': 'DisGeNET',
   'knowledge_source_type': 'knowledgebase',
 

In [36]:
jsonschema.validate(instance=tonsoref, schema=schema_from_yaml)
## yesyesyesyesyesesyseyseysys

In [37]:
## but what if it's a fluke?? 
## remove a required key from one association 
tonsoref['components']['x-bte-kgs-associations']['gene-disease2_disgenet-befree'].pop("category_measures_present")
## then try to validate, this 
jsonschema.validate(instance=tonsoref, schema=schema_from_yaml)
## so it works in catching the error yayyyyyyyy. too bad it doesn't tell you which entry had the error in it...

False

ValidationError: 'category_measures_present' is a required property

Failed validating 'required' in schema['properties']['components']['properties']['x-bte-kgs-associations']['patternProperties']['.']:
    {'properties': {'association_context': {'description': 'Python-dictionary-like. '
                                                           'Keys describe '
                                                           'what is the '
                                                           'type of '
                                                           'context. It '
                                                           'could be that '
                                                           'the '
                                                           'knowledge-source/association '
                                                           'was restricted '
                                                           'to a specific '
                               

In [38]:
tonsoref['components']['x-bte-kgs-associations']['gene-disease2_disgenet-befree']["category_measures_present"] = False

In [39]:
jsonschema.validate(instance=tonsoref, schema=schema_from_yaml)
## and it's back and accepted again 